In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
vocab = ["I","like","this","movie","very","much","!","nlp","is","great"]
embed_size = 5
vocab_size = 11 # 10个词，第一个位置留给补零的位置
class_num = 2

In [3]:
class TextCNN(nn.Module):
    def __init__(self, dropout=0.5):
        super(TextCNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) #词向量，可用word2vec
        self.conv4 = nn.Conv1d(in_channels=embed_size, out_channels=2, kernel_size=4, stride = 1)
        self.conv3 = nn.Conv1d(in_channels=embed_size, out_channels=2, kernel_size=3, stride = 1)
        self.conv2 = nn.Conv1d(in_channels=embed_size, out_channels=2, kernel_size=2, stride = 1)
        
        hidden_size = 2*3
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_size, class_num)
    
    def forward(self, x):
        x = self.embed(x) # [batch_size, seq_length]->[batch_size, seq_length,seq_embed_size]
#         print(x.size()) torch.Size([2, 6, 5])
        x = x.transpose(2,1) # [batch_size, seq_embed_size, seq_length]
#         print(x.size()) torch.Size([2, 5, 6]
        x1 = self.conv2(x) # [batch_size, out_channels, seq_length-k+l]
        
        x1 = F.relu(x1) # [batch_size, out_channels, seq_length-k+l]
        x1 = F.max_pool1d(x1, x1.size(2)) # [batch_size, out_channels]
        
        x2 = self.conv3(x)
        x2 = F.relu(x2)
        x2 = F.max_pool1d(x2, x2.size(2))
        
        x3 = self.conv4(x)
        x3 = F.relu(x3)
        x3 = F.max_pool1d(x3, x3.size(2))
        
        x = torch.cat((x1, x2, x3), 1) #[batch_size, out_channels*3 , 1]
        x = self.dropout(x)
        x = x.squeeze() #[batch_size, out_channels*3]
        out = self.linear(x)
        
        return out

In [4]:
train_text = [["I","like","this","movie","very","much","!"],
            ['nlp',"is","great"]]
label = [0,1]

In [5]:
word2id = {}
id2word = {}
for index,word in enumerate(vocab):
    word2id[word] = index+1
    id2word[index+1]=word

In [6]:
train_ids = [[word2id[w] for w in s] for s in train_text]
train_ids

[[1, 2, 3, 4, 5, 6, 7], [8, 9, 10]]

短的补0，过长的截断

In [7]:
max_length = 6
for i in range(len(train_ids)):
    s = train_ids[i]
    train_ids[i] = s[:max_length]+[0]*(max_length-len(s))
train_ids

[[1, 2, 3, 4, 5, 6], [8, 9, 10, 0, 0, 0]]

In [8]:
model = TextCNN()
optimizer = torch.optim.Adam(model.parameters())

In [9]:
train_tensor = torch.tensor(train_ids)
train_tensor.size()

torch.Size([2, 6])

In [10]:
label = torch.tensor(label)
label

tensor([0, 1])

In [11]:
model.train()

TextCNN(
  (embed): Embedding(11, 5)
  (conv4): Conv1d(5, 2, kernel_size=(4,), stride=(1,))
  (conv3): Conv1d(5, 2, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(5, 2, kernel_size=(2,), stride=(1,))
  (dropout): Dropout(p=0.5)
  (linear): Linear(in_features=6, out_features=2, bias=True)
)

In [12]:
for epoch in range(100):
    model.zero_grad()
    outputs = model(train_tensor)
    loss = F.cross_entropy(outputs, label)
    loss.backward()
    optimizer.step()
    print(loss.item())

0.6908247470855713
0.9275128841400146
0.8639361262321472
0.7146687507629395
0.5801581144332886
0.8843567967414856
0.8364002704620361
0.7429993152618408
0.6341226696968079
0.49596062302589417
0.8533338308334351
0.676983118057251
0.8743958473205566
0.4713447391986847
0.6170797348022461
0.7254218459129333
0.6794955730438232
0.8573406934738159
0.7313684225082397
0.6111850738525391
0.6743123531341553
0.699020266532898
0.518700897693634
0.6605005264282227
0.6464823484420776
0.7326514720916748
0.6220919489860535
0.49077868461608887
0.8059700131416321
0.3801306486129761
0.45203810930252075
0.4112955927848816
0.8763601183891296
0.3816373348236084
1.004064679145813
0.46701598167419434
0.6195912957191467
0.4708707332611084
0.5191164612770081
0.956315815448761
0.3962726593017578
0.48859870433807373
0.4357954263687134
0.6388231515884399
0.8236682415008545
0.7422412037849426
0.9560737013816833
0.6414955854415894
0.7805585861206055
0.8167399764060974
0.6091652512550354
0.7299103736877441
0.6220731139